# sklearn-porter

Repository: https://github.com/nok/sklearn-porter

## LinearSVC

Documentation: [sklearn.svm.LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)

### Loading data:

In [2]:
from sklearn.datasets import load_iris

iris_data = load_iris()
X = iris_data.data
y = iris_data.target

print(X.shape, y.shape)

((150, 4), (150,))


### Train classifier:

In [3]:
from sklearn import svm

clf = svm.LinearSVC(C=1., random_state=0)
clf.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

### Transpile classifier:

In [5]:
from sklearn_porter import Porter

porter = Porter(clf, language='go')
output = porter.export()

print(output)

package main

import (
	"os"
	"fmt"
	"strconv"
	"math"
)

type LinearSVC struct {
	coefficients [][]float64
	intercepts []float64
}

func (svc LinearSVC) predict(features []float64) int {
	classIdx := 0
	classVal := math.Inf(-1)
	outerCount, innerCount := len(svc.coefficients), len(svc.coefficients[0])
	for i := 0; i < outerCount; i++ {
		var prob float64
		for j := 0; j < innerCount; j++ {
			prob = prob + svc.coefficients[i][j] * features[j]
		}
		if prob + svc.intercepts[i] > classVal {
			classVal = prob + svc.intercepts[i]
			classIdx = i
		}
	}
	return classIdx
}

func main() {

	// Features:
	var features []float64
	for _, arg := range os.Args[1:] {
		if n, err := strconv.ParseFloat(arg, 64); err == nil {
			features = append(features, n)
		}
	}

	// Parameters:
	coefficients := [][]float64{{0.18424209458473811, 0.45123000025163923, -0.80794587716737576, -0.45071660033253858}, {0.052877455748516447, -0.89214995228605254, 0.40398084459610972, -0.9376821661447452}, {-0.85070784319